<a href="https://colab.research.google.com/github/dfragos/EMOS-2024/blob/main/Spacial/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install R and IRkernel
!apt-get install r-base
!R -e "install.packages('IRkernel', repos='http://cran.us.r-project.org')"
!R -e "IRkernel::installspec(user = FALSE)"

Colab menu: Runtime > Change runtime type.
Runtime type: R.

In [ ]:
rm(list=ls())

# load the libraries with all dependencies
if(!require('giscoR')) install.packages('giscoR', dependencies=T); library('giscoR')

In [ ]:
library(giscoR)
library(sf)
library(dplyr)

# Different resolutions
GRC_res60 <- gisco_get_countries(resolution = "60", country = "GRC") %>%
  mutate(res = "60M")
GRC_res20 <-
  gisco_get_countries(resolution = "20", country = "GRC") %>%
  mutate(res = "20M")
GRC_res10 <-
  gisco_get_countries(resolution = "10", country = "GRC") %>%
  mutate(res = "10M")
GRC_res03 <-
  gisco_get_countries(resolution = "03", country = "GRC") %>%
  mutate(res = "03M")


GRC_all <- bind_rows(GRC_res60, GRC_res20, GRC_res10, GRC_res03)

# Plot ggplot2

library(ggplot2)

ggplot(GRC_all) +
  geom_sf(fill = "tomato") +
  facet_wrap(vars(res)) +
  theme_minimal()

In [ ]:


# Labels and Lines available

labs <- gisco_get_countries(
  spatialtype = "LB",
  region = "Africa",
  epsg = "3857"
)

coast <- gisco_get_countries(
  spatialtype = "COASTL",
  epsg = "3857"
)

# For zooming
afr_bbox <- st_bbox(labs)

ggplot(coast) +
  geom_sf(col = "deepskyblue4", linewidth = 3) +
  geom_sf(data = labs, fill = "springgreen4", col = "darkgoldenrod1", size = 5, shape = 21) +
  coord_sf(
    xlim = afr_bbox[c("xmin", "xmax")],
    ylim = afr_bbox[c("ymin", "ymax")]
  )

In [ ]:
# Get shapes
nuts3 <- gisco_get_nuts(
  year = "2021",
  epsg = "3035",
  resolution = "10",
  nuts_level = "3"
)

# Group by NUTS by country and convert to lines
country_lines <- nuts3 %>%
  group_by(
    CNTR_CODE
  ) %>%
  summarise(n = n()) %>%
  st_cast("MULTILINESTRING")